In [1]:
from haystack.telemetry import tutorial_running
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
import logging
from haystack.nodes import PreProcessor
from langdetect import detect
import nltk



c:\Users\sotof\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Logs
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
# all_docs = convert_files_to_docs(dir_path=doc_dir)

converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path="prueba.txt", meta=None)[0]

In [9]:
from haystack.utils import convert_files_to_docs


all_docs = convert_files_to_docs(dir_path='tesis-qa-system')

In [4]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sotof\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 66.56docs/s]

n_docs_input: 1
n_docs_output: 2


In [5]:
# Not respecting sentence boundary vs respecting sentence boundary

preprocessor_nrsb = PreProcessor(split_respect_sentence_boundary=False)
docs_nrsb = preprocessor_nrsb.process([doc_txt])

print("RESPECTING SENTENCE BOUNDARY")
end_text = docs_default[0].content[-50:]
print('End of document: "...' + end_text + '"')
print()
print("NOT RESPECTING SENTENCE BOUNDARY")
end_text_nrsb = docs_nrsb[0].content[-50:]
print('End of document: "...' + end_text_nrsb + '"')

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 663.13docs/s]

RESPECTING SENTENCE BOUNDARY
End of document: "..., is included as an eligible practice under EQIP. "

NOT RESPECTING SENTENCE BOUNDARY
End of document: "...EQIP funding is reserved for livestock operations."


In [6]:
# Sliding window approach

preprocessor_sliding_window = PreProcessor(split_overlap=3, split_length=10, split_respect_sentence_boundary=False)
docs_sliding_window = preprocessor_sliding_window.process([doc_txt])

doc1 = docs_sliding_window[0].content[:200]
doc2 = docs_sliding_window[1].content[:100]
doc3 = docs_sliding_window[2].content[:100]

print('Document 1: "' + doc1 + '..."')
print('Document 2: "' + doc2 + '..."')
print('Document 3: "' + doc3 + '..."')

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 500.39docs/s]

Document 1: "The EQIP covers up to 75 percent of costs incurred..."
Document 2: "of costs incurred and 100 percent of income foregone for..."
Document 3: "income foregone for implementing eligible conservation practices for awardees. There..."


In [11]:
all_docs = convert_files_to_docs(dir_path='tesis-qa-system')
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

Preprocessing: 0docs [00:00, ?docs/s]

n_files_input: 0
n_docs_output: 0
